## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps 
gmaps.configure(api_key=g_key)

In [8]:
# 1-4
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Marsaxlokk,MT,35.8419,14.5431,81.09,78,0,11.77,clear sky
1,1,Hervey Bay,AU,-25.2986,152.8535,68.00,81,100,10.25,overcast clouds
2,2,Taoudenni,ML,22.6783,-3.9836,95.88,15,2,4.23,clear sky
3,3,Jamestown,US,42.0970,-79.2353,64.18,64,0,5.75,clear sky
4,4,Barrow,US,71.2906,-156.7887,37.42,87,75,17.27,mist


In [10]:
# 5 Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


In [11]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]


In [12]:
# Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                356
City                   356
Country                350
Lat                    356
Lng                    356
Max Temp               356
Humidity               356
Cloudiness             356
Wind Speed             356
Current Description    356
dtype: int64

In [13]:
#Drop any empty rows and create a DataFrame that doesn’t have empty rows.
clean_travel_df = preferred_cities_df.dropna()


In [14]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

#  Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hervey Bay,AU,68.00,overcast clouds,-25.2986,152.8535,
3,Jamestown,US,64.18,clear sky,42.0970,-79.2353,
8,Mogok,MM,68.95,overcast clouds,22.9167,96.5000,
9,Cidreira,BR,65.14,overcast clouds,-30.1811,-50.2056,
10,Prado,BR,68.94,clear sky,-17.3411,-39.2208,
14,Mataura,NZ,51.48,clear sky,-46.1927,168.8643,
16,Leh,IN,59.59,overcast clouds,34.1667,77.5833,
17,Pocos De Caldas,BR,65.50,scattered clouds,-21.7878,-46.5614,
19,Cape Town,ZA,60.39,clear sky,-33.9258,18.4232,
21,Rocha,UY,59.65,overcast clouds,-34.4833,-54.3333,


In [15]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():

    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"

    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(str(index) + ": Hotel Name in <" + hotel_df["City"] + "> not found... skipping.")
        pass
        

1      1: Hotel Name in <Hervey Bay> not found... ski...
3      1: Hotel Name in <Jamestown> not found... skip...
8        1: Hotel Name in <Mogok> not found... skipping.
9      1: Hotel Name in <Cidreira> not found... skipp...
10       1: Hotel Name in <Prado> not found... skipping.
                             ...                        
689    1: Hotel Name in <Fougamou> not found... skipp...
693    1: Hotel Name in <Chegdomyn> not found... skip...
694    1: Hotel Name in <Chumayel> not found... skipp...
697    1: Hotel Name in <Pontal Do Parana> not found....
700      1: Hotel Name in <Maloy> not found... skipping.
Name: City, Length: 350, dtype: object
1      3: Hotel Name in <Hervey Bay> not found... ski...
3      3: Hotel Name in <Jamestown> not found... skip...
8        3: Hotel Name in <Mogok> not found... skipping.
9      3: Hotel Name in <Cidreira> not found... skipp...
10       3: Hotel Name in <Prado> not found... skipping.
                             ...                 

In [16]:
# Drop the rows where there is no Hotel Name

clean_hotel_df = hotel_df[(hotel_df["Hotel Name"] != "")]
clean_hotel_df = clean_hotel_df.head(180)

# Confirm the data looks good
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [17]:
## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
locations = clean_hotel_df[["Lat", "Lng"]]


In [22]:
# Create Figure
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, -35.0), zoom_level=2.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display the figure
fig


AttributeError: module 'collections' has no attribute 'Iterable'